# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [7]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vipinvijayan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/vipinvijayan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [8]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [9]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [10]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Use-Case Data!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [11]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [12]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [13]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [14]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 64, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [15]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]           unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

KnowledgeGraph(nodes: 86, relationships: 710)

We can save and load our knowledge graphs as follows.

In [16]:
kg.save("usecase_data_kg.json")
usecase_data_kg = KnowledgeGraph.load("usecase_data_kg.json")
usecase_data_kg

KnowledgeGraph(nodes: 86, relationships: 710)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [17]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=usecase_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [18]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


Finally, we can use our `TestSetGenerator` to generate our testset!

In [19]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating Samples: 100%|██████████| 10/10 [00:03<00:00,  2.78it/s]


,user_input,reference_contexts,reference,synthesizer_name
0,What insights does Acemoglu (2024) provide reg...,[Introduction ChatGPT launched in November 202...,The context mentions that Acemoglu (2024) is o...,single_hop_specifc_query_synthesizer
1,What is the significance of June 2024 in ChatG...,[Table 1: ChatGPT daily message counts (millio...,"The report provides data ending on June 26th, ...",single_hop_specifc_query_synthesizer
2,How is ChatGPT usage distributed among differe...,[Variation by Occupation Figure 23 presents va...,Variation by occupation Figure 23 presents var...,single_hop_specifc_query_synthesizer
3,How does the concept of Writting relate to the...,[Conclusion This paper studies the rapid growt...,Writing is by far the most common work use of ...,single_hop_specifc_query_synthesizer
4,How do usage patterns and growht diffrent amon...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,The context indicates that ChatGPT launched in...,multi_hop_abstract_query_synthesizer
5,How does the privacy-preserving methodology us...,[<1-hop>\n\nConclusion This paper studies the ...,The privacy-preserving methodology introduced ...,multi_hop_abstract_query_synthesizer
6,how user behavior and usage patterns changed f...,[<1-hop>\n\nMonth Non-Work (M) (%) Work (M) (%...,"from june 2024 to june 2025, the total message...",multi_hop_abstract_query_synthesizer
7,what happened in july 2025 with chatgpt usage ...,[<1-hop>\n\nConclusion This paper studies the ...,"In july 2025, chatgpt had over 700 million use...",multi_hop_specific_query_synthesizer
8,how do 700 million users use chatgpt for work ...,[<1-hop>\n\nConclusion This paper studies the ...,"According to the context, by July 2025, ChatGP...",multi_hop_specific_query_synthesizer
9,Based on the data indicating that 18 billion m...,[<1-hop>\n\nConclusion This paper studies the ...,"The context shows that by July 2025, ChatGPT u...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [20]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]           unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

In [21]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What Reuters do I need know about ChatGPT?,[Introduction ChatGPT launched in November 202...,"The context discusses ChatGPT, a mass-market c...",single_hop_specifc_query_synthesizer
1,How is ChatGPT used for seeking info?,[Table 1: ChatGPT daily message counts (millio...,ChatGPT is used for seeking information about ...,single_hop_specifc_query_synthesizer
2,What does the 'Section: Data and Privacy' cove...,[Figure 23 presents variation in ChatGPT usage...,The 'Section: Data and Privacy' discusses vari...,single_hop_specifc_query_synthesizer
3,How does management influence ChatGPT usage?,[Conclusion This paper studies the rapid growt...,The context does not provide specific informat...,single_hop_specifc_query_synthesizer
4,How do user behavior and usage trends over tim...,[<1-hop>\n\nTable 1: ChatGPT daily message cou...,The data shows that while total ChatGPT messag...,multi_hop_abstract_query_synthesizer
5,how does prvacy-preserving methdology impact g...,[<1-hop>\n\nConclusion This paper studies the ...,The paper introduces a novel privacy-preservin...,multi_hop_abstract_query_synthesizer
6,H0w do regression analisis of message shares s...,[<1-hop>\n\nFigure 23 presents variation in Ch...,The context indicates that users in highly pai...,multi_hop_abstract_query_synthesizer
7,How does the message volume comparison between...,[<1-hop>\n\nMonth Non-Work (M) (%) Work (M) (%...,"In Jun 2024, non-work messages accounted for 5...",multi_hop_abstract_query_synthesizer
8,How does the usage of ChatGPT by Claude compar...,[<1-hop>\n\nTable 1: ChatGPT daily message cou...,"The data indicates that ChatGPT usage, includi...",multi_hop_specific_query_synthesizer
9,How does OpenAI's development and widespread a...,[<1-hop>\n\nTable 1: ChatGPT daily message cou...,"The context shows that OpenAI's ChatGPT, launc...",multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [22]:
from langsmith import Client

client = Client()

dataset_name = "Use Case Synthetic Data - AIE8"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Synthetic Data for Use Cases"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [23]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [24]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [26]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [27]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG"
)

In [28]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [29]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

As is usual: We'll be using `gpt-4.1-mini` for our RAG!

In [30]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [31]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [32]:
rag_chain.invoke({"question" : "What are people doing with AI these days?"})

'Based on the provided context, people are using AI, particularly generative AI like ChatGPT, for a wide range of activities both at work and outside of work. These uses include:\n\n- Performing workplace tasks by either augmenting or automating human labor.  \n- Producing digital products such as writing, software code, and spreadsheets.  \n- Seeking information and advice, similar to but more flexibly than traditional web search engines.  \n- Using AI as co-workers that produce output or as co-pilots that give advice and improve productivity.  \n- Engaging in activities related to self-expression, such as relationships, personal reflection, games, and role play (though these are smaller proportions of usage).  \n- Therapy and companionship as noted by some estimates.\n\nIn summary, people are leveraging AI to enhance productivity, create digital content, get advice, automate tasks, and also for personal and social uses.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [33]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [34]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dopeness_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this response dope, lit, cool, or is it just a generic response?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `dopeness_evaluator`:

## LangSmith Evaluation

In [35]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'clear-machine-41' at:
https://smith.langchain.com/o/746284f9-bb47-4300-a64c-6b47652078eb/datasets/8b79da12-0128-4443-9c5d-7a0dc5735eb5/compare?selectedSessions=d0d48be8-d688-4192-b09b-e6109df388d6




12it [03:44, 18.70s/it]


,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,Considering the extensive growth of ChatGPT us...,"Based on the provided context, OpenAI's develo...",None,The context indicates that OpenAI has develope...,1,1,0,9.375351,9c609054-a457-4296-8815-6dbcf5a00e5b,2456b3ae-ab6f-4e95-a781-3cf07dce468b
1,How does OpenAI's development and widespread a...,OpenAI's development and widespread adoption o...,None,The context highlights that OpenAI launched Ch...,1,1,0,5.583989,38f0d98d-ab6d-4d31-853d-09f6ed9554a1,a4deaaf4-74a1-4896-9b0d-1e41757215dc
2,How does OpenAI's development and widespread a...,OpenAI's development and widespread adoption o...,None,"The context shows that OpenAI's ChatGPT, launc...",1,1,0,4.752205,32017e6d-b781-42eb-8eff-dcfbaca65695,ddc010df-1034-4087-b1ec-0df2d05b3cad
3,How does the usage of ChatGPT by Claude compar...,"Based on the provided context, the comparison ...",None,"The data indicates that ChatGPT usage, includi...",0,0,0,6.707233,a6f5e174-3954-4236-8604-241ebab3338e,892355b2-23a2-49c9-a98c-9204431b3c76
4,How does the message volume comparison between...,Based on the context provided:\n\n- **Message ...,None,"In Jun 2024, non-work messages accounted for 5...",1,1,0,8.449623,c0439cee-d124-469e-9fdd-42cca15d7018,a31448d1-9983-4a02-8e27-66e224e0fa44
5,H0w do regression analisis of message shares s...,"Based on the provided context, regression anal...",None,The context indicates that users in highly pai...,1,1,0,2.616382,0beeed7a-9990-4414-a597-323c92254152,d0bcba06-6bbf-471b-acd1-e27361db3212
6,how does prvacy-preserving methdology impact g...,The provided context does not explicitly expla...,None,The paper introduces a novel privacy-preservin...,0,0,0,1.961433,979cab08-90bd-4f03-afc4-ed9dd0665396,9557d761-e48a-4d87-92bc-f7a14ca516c2
7,How do user behavior and usage trends over tim...,User behavior and usage trends over time show ...,None,The data shows that while total ChatGPT messag...,1,1,0,4.465862,33b6fd05-dbae-4b82-8c47-f21ff363d54c,5b089e59-eae9-46a9-bd5f-496320543899
8,How does management influence ChatGPT usage?,I don't know.,None,The context does not provide specific informat...,1,0,0,1.481599,534344a1-5335-422e-9336-3883fa652a15,b139c091-9f84-4a67-864a-40b392da673f
9,What does the 'Section: Data and Privacy' cove...,I don't know.,None,The 'Section: Data and Privacy' discusses vari...,0,0,0,0.812359,903638b6-e5bc-448e-8287-22bbc9863a69,b9676587-1f49-4f37-8103-23869a029974


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [36]:
DOPENESS_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Make your answer rad, ensure high levels of dopeness. Do not be generic, or give generic responses.

Context: {context}
Question: {question}
"""

dopeness_rag_prompt = ChatPromptTemplate.from_template(DOPENESS_RAG_PROMPT)

In [37]:
rag_documents = docs

In [38]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

In [39]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

In [40]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG Docs"
)

In [41]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [42]:
dopeness_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dopeness_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [43]:
dopeness_rag_chain.invoke({"question" : "How are people using AI to make money?"})

'Alright, let’s blast off with some AI-powered goldmining insight straight from the brainy cosmos of this data!\n\nPeople ain’t just asking AI to spit out answers—they’re surfing the turbo wave of generative AI like ChatGPT to **supercharge their hustle**. The docs paint a slick picture: folks are using AI less as a drudge-worker and more like a wizardly advisor or research assistant. It’s like having a decision-making sidekick that pumps up the quality of your choices, especially in knowledge-heavy gigs. By dialing up decision support, AI cranks worker output to eleven, boosting productivity through sharper, faster, smarter moves.\n\nThis means people are grinding smarter, not harder, leaning on AI to augment, automate, or outright elevate tasks—whether it’s crafting killer content, gathering strategic intel, or navigating complex problem-solving. The economic mojo here? The US alone boasts a mind-blowing **$97 billion consumer surplus** from AI usage—people value this digital sidekic

Finally, we can evaluate the new chain on the same test set!

In [ ]:
evaluate(
    dopeness_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "dopeness_rag_chain"},
)

View the evaluation results for experiment: 'shiny-push-49' at:
https://smith.langchain.com/o/746284f9-bb47-4300-a64c-6b47652078eb/datasets/8b79da12-0128-4443-9c5d-7a0dc5735eb5/compare?selectedSessions=3a39210b-0533-4750-9e47-ee971b1af9be




0it [00:00, ?it/s]

#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.